In [1]:
# Importa os módulos para a análise dos dados
%matplotlib
import numpy
import pandas

pandas.options.mode.chained_assignment = None

Using matplotlib backend: module://matplotlib_inline.backend_inline


In [2]:
SEMANA = 'Domingo Segunda Terça VILLA Quinta Sexta Sábado'.split(' ')
WEEK = {
    'Monday': 'Segunda', 
    'Tuesday': 'Terça', 
    'Wednesday': 'VILLA', 
    'Thursday': 'Quinta',
    'Friday': 'Sexta',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

# Derivei a fórmula 4n-2[n/3] pra explicar a progressão 1 é 4, 3 é 10. Tenho muito tempo livre.
# Como a função floor não tem inverso, tive que fazer uma tabela de valores
VALORES = [(4*n - 2*int(n/3)) for n in range(1, 20)]

BEBIDAS = {
    'litrao': 1000, # mL
    'petra': 269, # mL
    'corote': 500, # mL
    'chopp': 0 # verificar mL do copo
}

def inicializar_entradas(csv: str, threshold: float) -> pandas.core.frame.DataFrame:
    # Importa o extrato completo
    extrato = pandas.read_csv(csv)
    
    # Converte o valor dos pix pra float
    extrato['Valor'] = extrato['Valor'].str.replace(',', '')
    extrato['Valor'] = extrato['Valor'].astype(float)

    # Mantém apenas os pix de entrada e cujo valor esteja abaixo de 100
    entradas = extrato[extrato['Valor'] > 0]
    entradas = entradas[entradas['Valor'] < threshold]

    # Converte as datas para o tipo datetime e organiza em ordem ascendente
    entradas['Data Lançamento'] = pandas.to_datetime(entradas['Data Lançamento'], dayfirst=True).dt.normalize()
    entradas = entradas.sort_values(by='Data Lançamento', ascending=True)

    # Adiciona uma coluna para os dias da semana
    entradas.loc[:, 'Dia'] = entradas['Data Lançamento'].dt.day_name()
    entradas['Dia'] = entradas['Dia'].replace(WEEK)

    # Normaliza todos os nomes para uppercase
    entradas['Nome'] = entradas['Nome'].str.upper()

    return entradas

In [3]:
# Converte o pix em mL de bebida consumida
def pix_to_mL(pix):
    # Verifica se é litrão
    if pix == 15:
        return 1000
 
    return 269 * (VALORES.index(pix) + 1)

# Converte a entrada em pix em mL de EtOH consumido com base no teor alcoólico
def pix_to_EtOH(pix):
    # Verifica se é litrão
    if pix == 15:
        return 1000 * 5/100

    return (269 * (VALORES.index(pix) + 1)) * 4.8/100

In [4]:
# Calcula o consumo de álcool/bebida para uma pessoa
def calcular_alcoolismo(extrato, nome, func):
    # Filtra o extrato para incluir apenas a pessoa e os valores válidos
    pessoa = extrato[extrato['Nome'] == nome]
    pessoa = pessoa[pessoa['Valor'].isin(VALORES+[15])]

    # Calculo e retorna o consumo total
    consumo = pessoa.groupby('Valor')['Valor'].transform(lambda x: x.apply(func)).sum()
    return consumo

In [5]:
def alcoolismo_global(extrato, func):
    # Filtra apenas os pix que já foram validados
    pix_validos = extrato[extrato['Valor'].isin(VALORES + [15])]

    # Cria uma nova coluna calculando o consumo por transferência
    pix_validos.loc[:, 'Consumo (mL)'] = pix_validos['Valor'].transform(lambda x: x.apply(func))

    # Agrupa e soma o consumo total de cada pessoa
    consumo = pix_validos.groupby('Nome')['Consumo (mL)'].sum()
    consumo = consumo.sort_values(ascending=False)

    return consumo